*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.6.0-dev20210530


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2021-05-30 16:26:13--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  --.-KB/s    in 0.007s  

2021-05-30 16:26:13 (52.5 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2021-05-30 16:26:13--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[====

In [ ]:
#data loading and cleaning if required
dataset = pd.read_csv(train_file_path, sep ='\t',names=["label", "message"])
dataset.head()


dataset.isnull().sum()
dataset.groupby('label').describe()

dataset["label"]=dataset["label"].map({'spam':1,'ham':0})
dataset.head()
dataset['message']


0       ahhhh...just woken up!had a bad dream about u ...
1                                you can never do nothing
2       now u sound like manky scouse boy steve,like! ...
3       mum say we wan to go then go... then she can s...
4       never y lei... i v lazy... got wat? dat day ü ...
                              ...                        
4174    just woke up. yeesh its late. but i didn't fal...
4175    what do u reckon as need 2 arrange transport i...
4176    free entry into our £250 weekly competition ju...
4177    -pls stop bootydelious (32/f) is inviting you ...
4178    tell my  bad character which u dnt lik in me. ...
Name: message, Length: 4179, dtype: object

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense, Dropout


traindata, testdata, trainlabels, testlabels = train_test_split(dataset['message'], dataset['label'], test_size=0.2)

token = Tokenizer(num_words = 500)
token.fit_on_texts(traindata)

seq = tokenizer.texts_to_sequences(traindata)
pad = pad_sequences(seq, maxlen = 150,padding="post",truncating="post")

testing_seq = tokenizer.texts_to_sequences(testdata)
testing_pad = pad_sequences(testing_seq, maxlen = max_len,padding = "post", truncating = "post")




In [ ]:

model = Sequential()
model.add(Embedding(500, 16, input_length=150))
model.add(GlobalAveragePooling1D())
model.add(Dense(24, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.summary()

model.compile(loss='binary_crossentropy',optimizer='adam' ,metrics=['accuracy'])

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 150, 16)           8000      
_________________________________________________________________
global_average_pooling1d_6 ( (None, 16)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 24)                408       
_________________________________________________________________
dropout_6 (Dropout)          (None, 24)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 25        
Total params: 8,433
Trainable params: 8,433
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(pad, trainlabels, epochs=50, validation_data=(testing_pad, testlabels), verbose=2)
model.evaluate(testing_pad, testlabels)


Epoch 1/50
105/105 - 1s - loss: 0.5847 - accuracy: 0.8504 - val_loss: 0.4686 - val_accuracy: 0.8505
Epoch 2/50
105/105 - 0s - loss: 0.4008 - accuracy: 0.8699 - val_loss: 0.3543 - val_accuracy: 0.8505
Epoch 3/50
105/105 - 0s - loss: 0.3747 - accuracy: 0.8699 - val_loss: 0.3139 - val_accuracy: 0.8505
Epoch 4/50
105/105 - 0s - loss: 0.3560 - accuracy: 0.8699 - val_loss: 0.2924 - val_accuracy: 0.8505
Epoch 5/50
105/105 - 0s - loss: 0.3398 - accuracy: 0.8699 - val_loss: 0.2917 - val_accuracy: 0.8505
Epoch 6/50
105/105 - 0s - loss: 0.3057 - accuracy: 0.8699 - val_loss: 0.2922 - val_accuracy: 0.8505
Epoch 7/50
105/105 - 0s - loss: 0.2579 - accuracy: 0.8759 - val_loss: 0.2682 - val_accuracy: 0.8911
Epoch 8/50
105/105 - 0s - loss: 0.1956 - accuracy: 0.9267 - val_loss: 0.2793 - val_accuracy: 0.8565
Epoch 9/50
105/105 - 0s - loss: 0.1490 - accuracy: 0.9539 - val_loss: 0.3270 - val_accuracy: 0.8553
Epoch 10/50
105/105 - 0s - loss: 0.1253 - accuracy: 0.9599 - val_loss: 0.3272 - val_accuracy: 0.8684

[0.28537702560424805, 0.9377990365028381]

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    pred_text = [pred_text]

    pred_seq = tokenizer.texts_to_sequences(pred_text)
    pred_pad = pad_sequences(pred_seq, maxlen =50,
                      padding = 'post',
                      truncating='post')
    pred = (model.predict(pred_pad))
    print(pred)
    if pred <0.5:
      result = [pred,'ham']
    else:
      result = [pred,'spam']
    return result

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[[0.00013906]]
[array([[0.00013906]], dtype=float32), 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


[[0.00013906]]
[[0.9994533]]
[[3.2446717e-06]]
[[1.]]
[[1.]]
[[4.3640202e-07]]
[[0.00012317]]
You passed the challenge. Great job!
